# [Polynomial Expansion for Orientation and Motion Estimation](http://www.diva-portal.org/smash/get/diva2:302485/FULLTEXT01.pdf)

Gunnar Farnebäck

## 4. Polynomial Expansion

Based on example signal and applicability from chapter 3.4

**Computes the polynomial expansion on the whole signal through consecutive 1D cross correlations**

In [1]:
import numpy as np
import scipy

**Marked point**

In [2]:
point = (2, 4)

**Signal**

In [3]:
f = np.array([[3, 7, 4, 5, 8],
              [9, 2, 4, 4, 6],
              [5, 1, 4, 3, 7],
              [3, 1, 1, 2, 8],
              [4, 6, 2, 3, 6],
              [7, 3, 2, 6, 3],
              [9, 6, 4, 9, 9]])

**Applicability**

In [4]:
a = np.array([1, 2, 1])

The applicability fixes the size of the neighborhood

In [5]:
n = int(a.size / 2)

**Polynomial basis**

{1, x, y, x², y², xy}

In [6]:
ones = np.ones(a.size, dtype=int)
x = np.arange(-n, n + 1, dtype=int)

B_x = np.stack([ones, x,    ones, x ** 2, ones,   x])
B_y = np.stack([ones, ones, x,    ones,   x ** 2, x])
B = np.einsum('ki,kj->ijk', B_x, B_y).reshape(a.size ** 2, B_x.shape[0])

print(B)

[[ 1 -1 -1  1  1  1]
 [ 1 -1  0  1  0  0]
 [ 1 -1  1  1  1 -1]
 [ 1  0 -1  0  1  0]
 [ 1  0  0  0  0  0]
 [ 1  0  1  0  1  0]
 [ 1  1 -1  1  1 -1]
 [ 1  1  0  1  0  0]
 [ 1  1  1  1  1  1]]


In [7]:
G = np.dot(np.outer(a, a).reshape(-1) * B.T, B)
invG = np.linalg.inv(G)

print(G, end='\n\n')
print(invG)

[[16  0  0  8  8  0]
 [ 0  8  0  0  0  0]
 [ 0  0  8  0  0  0]
 [ 8  0  0  8  4  0]
 [ 8  0  0  4  8  0]
 [ 0  0  0  0  0  4]]

[[ 0.1875  0.      0.     -0.125  -0.125   0.    ]
 [ 0.      0.125   0.      0.      0.      0.    ]
 [ 0.      0.      0.125   0.      0.      0.    ]
 [-0.125   0.      0.      0.25    0.      0.    ]
 [-0.125   0.      0.      0.      0.25    0.    ]
 [ 0.      0.      0.      0.      0.      0.25  ]]


In [8]:
BWa_1D_one = a
BWa_1D_x = a * x
BWa_1D_x2 = a * x ** 2

print(np.stack([BWa_1D_one, BWa_1D_x, BWa_1D_x2]))

[[ 1  2  1]
 [-1  0  1]
 [ 1  0  1]]


**Separable correlation**

x direction

In [9]:
BWaf_1D_one = scipy.ndimage.correlate1d(f, BWa_1D_one, axis=1, mode='constant')
BWaf_1D_x = scipy.ndimage.correlate1d(f, BWa_1D_x, axis=1, mode='constant')
BWaf_1D_x2 = scipy.ndimage.correlate1d(f, BWa_1D_x2, axis=1, mode='constant')

print(BWaf_1D_one, end='\n\n')
print(BWaf_1D_x, end='\n\n')
print(BWaf_1D_x2)

[[13 21 20 22 21]
 [20 17 14 18 16]
 [11 11 12 17 17]
 [ 7  6  5 13 18]
 [14 18 13 14 15]
 [17 15 13 17 12]
 [24 25 23 31 27]]

[[ 7  1 -2  4 -5]
 [ 2 -5  2  2 -4]
 [ 1 -1  2  3 -3]
 [ 1 -2  1  7 -2]
 [ 6 -2 -3  4 -3]
 [ 3 -5  3  1 -6]
 [ 6 -5  3  5 -9]]

[[ 7  7 12 12  5]
 [ 2 13  6 10  4]
 [ 1  9  4 11  3]
 [ 1  4  3  9  2]
 [ 6  6  9  8  3]
 [ 3  9  9  5  6]
 [ 6 13 15 13  9]]


y direction

In [10]:
BWaf_one = scipy.ndimage.correlate1d(BWaf_1D_one, BWa_1D_one.T, axis=0, mode='constant')
BWaf_x = scipy.ndimage.correlate1d(BWaf_1D_x, BWa_1D_one.T, axis=0, mode='constant')
BWaf_y = scipy.ndimage.correlate1d(BWaf_1D_one, BWa_1D_x.T, axis=0, mode='constant')
BWaf_x2 = scipy.ndimage.correlate1d(BWaf_1D_x2, BWa_1D_one.T, axis=0, mode='constant')
BWaf_y2 = scipy.ndimage.correlate1d(BWaf_1D_one, BWa_1D_x2.T, axis=0, mode='constant')
BWaf_xy = scipy.ndimage.correlate1d(BWaf_1D_x, BWa_1D_x.T, axis=0, mode='constant')

BWaf = np.stack([BWaf_one, BWaf_x, BWaf_y, BWaf_x2, BWaf_y2, BWaf_xy])

print(BWaf[:, point[1], point[0]])

[44 -2  8 30 18  2]


**Resulting coefficients**

In [11]:
r = np.einsum('li,ijk', invG, BWaf)
print(r[point[1], point[0]])

[ 2.25 -0.25  1.    2.   -1.    0.5 ]


**Reconstructed signal projection**

In [12]:
Br = np.einsum('ij,klj->kli', B, r)
print(Br[point[1], point[0]].reshape(a.size, a.size).T)

[[3.   0.25 1.5 ]
 [4.5  2.25 4.  ]
 [4.   2.25 4.5 ]]
